In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [3]:
df_green = spark.read.parquet('data/pq/green/*/*')

In [4]:
df_green.registerTempTable('green')

c:\Users\User\anaconda3\envs\data_engineering_camp\lib\site-packages\pyspark\sql\dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [5]:
df_green_revenue = spark.sql("""
SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

In [6]:
df_green_revenue.show()

+-------------------+----+------------------+--------------+
|               hour|zone|            amount|number_records|
+-------------------+----+------------------+--------------+
|2020-01-27 08:00:00|  91|123.88999999999999|             3|
|2020-01-03 13:00:00|  85|             38.55|             2|
|2020-01-08 22:00:00| 223|            144.58|            10|
|2020-01-06 09:00:00|  92|            183.34|             8|
|2020-01-17 21:00:00| 260|265.19000000000005|            22|
|2020-01-11 14:00:00| 243|178.79000000000002|             7|
|2020-01-19 16:00:00|  82| 611.6699999999998|            53|
|2020-01-27 10:00:00| 244|482.82000000000016|            22|
|2020-01-03 09:00:00|  74|1036.3099999999993|            87|
|2020-01-20 17:00:00| 166| 515.9300000000002|            33|
|2020-01-02 10:00:00|  74| 759.6499999999995|            65|
|2020-01-22 07:00:00| 223|            115.16|             5|
|2020-01-26 09:00:00|  35|            156.63|             5|
|2020-01-22 07:00:00|  7

In [7]:
df_green_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/green', mode='overwrite')

### Each partition has a result. After each "intermediate result" we have a "reshuffling" where the data is combined into one.

![images](../images/groupby.PNG)

In [8]:
df_yellow = spark.read.parquet('data/pq/yellow/*/*')
df_yellow.registerTempTable('yellow')

c:\Users\User\anaconda3\envs\data_engineering_camp\lib\site-packages\pyspark\sql\dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [9]:
df_yellow_revenue = spark.sql("""
SELECT 
    date_trunc('hour', tpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    yellow
WHERE
    tpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

In [10]:
df_yellow_revenue.show()

+-------------------+----+------------------+--------------+
|               hour|zone|            amount|number_records|
+-------------------+----+------------------+--------------+
|2020-01-03 23:00:00|  48|  8532.85000000002|           497|
|2020-01-23 19:00:00| 162|14219.900000000043|           792|
|2020-01-31 14:00:00| 237|11852.900000000041|           825|
|2020-01-23 14:00:00| 137|2787.6499999999987|           160|
|2020-01-01 15:00:00|  68|3559.2099999999973|           216|
|2020-01-01 00:00:00| 170| 8266.010000000015|           463|
|2020-01-04 00:00:00| 229|1935.0099999999989|           109|
|2020-01-23 11:00:00| 113|2094.4799999999987|           144|
|2020-01-23 11:00:00| 142| 4758.379999999999|           325|
|2020-01-11 11:00:00| 125| 787.6900000000003|            56|
|2020-01-22 07:00:00|  74|           1092.06|            82|
|2020-01-17 10:00:00| 170|  6841.69000000001|           418|
|2020-01-04 20:00:00| 161| 5983.240000000008|           373|
|2020-01-22 23:00:00|  7

In [11]:
df_yellow_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/yellow', mode='overwrite')

## Joining part

In [12]:
df_green_revenue = spark.read.parquet('data/report/revenue/green')
df_yellow_revenue = spark.read.parquet('data/report/revenue/yellow')

In [13]:
df_green_revenue_tmp = df_green_revenue \
    .withColumnRenamed('amount', 'green_amount') \
    .withColumnRenamed('number_records', 'green_number_records')

df_yellow_revenue_tmp = df_yellow_revenue \
    .withColumnRenamed('amount', 'yellow_amount') \
    .withColumnRenamed('number_records', 'yellow_number_records')

In [14]:
df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on=['hour', 'zone'], how='outer')

In [15]:
df_join.show()

+-------------------+----+------------------+--------------------+------------------+---------------------+
|               hour|zone|      green_amount|green_number_records|     yellow_amount|yellow_number_records|
+-------------------+----+------------------+--------------------+------------------+---------------------+
|2020-01-01 00:00:00|  10|              null|                null|             42.41|                    2|
|2020-01-01 00:00:00|  15|              null|                null|             34.09|                    1|
|2020-01-01 00:00:00|  24|              87.6|                   3| 754.9500000000003|                   45|
|2020-01-01 00:00:00|  32| 68.94999999999999|                   2|              18.0|                    1|
|2020-01-01 00:00:00|  42| 799.7599999999998|                  52| 635.3500000000003|                   46|
|2020-01-01 00:00:00|  55|129.29000000000002|                   4|              null|                 null|
|2020-01-01 00:00:00|  77| 7

In [16]:
df_join.write.parquet('data/report/revenue/total', mode='overwrite')

In [17]:
df_join = spark.read.parquet('data/report/revenue/total')

In [18]:
df_join

DataFrame[hour: timestamp, zone: int, green_amount: double, green_number_records: bigint, yellow_amount: double, yellow_number_records: bigint]

In [19]:
df_zones = spark.read.parquet('zones/')

In [20]:
df_result = df_join.join(df_zones, df_join.zone == df_zones.LocationID)

In [22]:
df_result.drop('LocationID', 'zone').write.parquet('tmp/revenue-zones')


In [23]:
df_result.show()

+-------------------+----+------------------+--------------------+------------------+---------------------+----------+---------+--------------------+------------+
|               hour|zone|      green_amount|green_number_records|     yellow_amount|yellow_number_records|LocationID|  Borough|                Zone|service_zone|
+-------------------+----+------------------+--------------------+------------------+---------------------+----------+---------+--------------------+------------+
|2020-01-01 00:00:00|   4|              null|                null|1004.3000000000003|                   57|         4|Manhattan|       Alphabet City| Yellow Zone|
|2020-01-01 00:00:00|  13|              null|                null|            1214.8|                   56|        13|Manhattan|   Battery Park City| Yellow Zone|
|2020-01-01 00:00:00|  45|              null|                null| 732.4800000000002|                   42|        45|Manhattan|           Chinatown| Yellow Zone|
|2020-01-01 00:00:00| 